# Analiza podataka - COVID-19

# **Uvoz neophodnih paketa**

---



In [ ]:
pip install --upgrade plotly

In [ ]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import folium

import math

# **Uvoz podataka i osnovne informacije**


---



In [ ]:
url="https://raw.githubusercontent.com/laxmimerit/Covid-19-Preprocessed-Dataset/master/preprocessed/covid_19_data_cleaned.csv"
df=pd.read_csv(url, parse_dates=['Date'])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape 

In [ ]:
df.describe()

In [ ]:
country_daywise =pd.read_csv('https://raw.githubusercontent.com/laxmimerit/Covid-19-Preprocessed-Dataset/master/preprocessed/country_daywise.csv',parse_dates=['Date'])

countrywise =pd.read_csv('https://raw.githubusercontent.com/laxmimerit/Covid-19-Preprocessed-Dataset/master/preprocessed/countrywise.csv')

daywise =pd.read_csv('https://raw.githubusercontent.com/laxmimerit/Covid-19-Preprocessed-Dataset/master/preprocessed/daywise.csv',parse_dates=['Date'])

In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.query('Country == "Serbia"')

# Ukupan broj potvrđenih, oporavljenih i umrlih u svetu

---



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=confirmed['Date'],y=confirmed['Confirmed'], mode='lines+markers', name='Confirmed',line=dict(color='Orange',width=2)))
fig.add_trace(go.Scatter(x=recovered['Date'],y=recovered['Recovered'], mode='lines+markers', name='Recovered',line=dict(color='Green',width=2)))
fig.add_trace(go.Scatter(x=deaths['Date'],y=deaths['Deaths'], mode='lines+markers', name='Deaths',line=dict(color='Red',width=2)))

fig.update_layout(title='<b>Covid-19 podaci</b>',title_x=0.5,xaxis_tickfont_size=14,yaxis=dict(title='Broj slučajeva'))

fig.show()

# Gustina slučajeva

---





In [ ]:
df['Date']=df['Date'].astype(str)

In [ ]:
fig = px.density_mapbox(df,lat = 'Lat', lon = 'Long' , hover_name = 'Country' , hover_data = ['Confirmed', 'Recovered', 'Deaths'], animation_frame='Date',color_continuous_scale= 'Portland',radius =7, zoom=0,height=700)
fig.update_layout(title='<b>Širenje infekcije Covid-19 tokom vremena</b>',title_x=0.5)
fig.update_layout(mapbox_style='open-street-map', mapbox_center_lon=0)
fig.show()


# Folium mape


---







In [ ]:
sample=df[df['Date']==max(df['Date'])]
sample

In [ ]:
mp=folium.Map(location=[0,0], tiles='cartodbpositron', min_zoom= 1, max_zoom=4, zoom_start=1)

for i in range(0, len(sample)):
  folium.Circle(location=[sample.iloc[i]['Lat'], sample.iloc[i]['Long']], color='crimson', fill='crimson',
                tooltip='<li><bold> Country:' + str(sample.iloc[i]['Country'])+
                        '<li><bold> Province:' + str(sample.iloc[i]['Province/State'])+
                          '<li><bold> Confirmed:' + str(sample.iloc[i]['Confirmed'])+
                            '<li><bold> Deaths:' + str(sample.iloc[i]['Deaths']),
                radius=int(sample.iloc[i]['Confirmed'])**0.5).add_to(mp)

mp

# Potvrdjeni slučajevi preko Choropleth Map


---



In [ ]:
fig=px.choropleth(country_daywise, locations='Country', locationmode='country names', color=np.log(country_daywise['Confirmed']),
                  hover_name='Country', animation_frame=country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                  title='<b>Slučajevi tokom vremena</b>', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale=True)
fig.update_layout(title_x=0.5)
fig.show()

# **Smrtni slučajevi i oporavljeni na 100 slučajeva**

---






In [ ]:
daywise.head()

In [ ]:
fig_c=px.bar(daywise, x='Date', y='Confirmed', color_discrete_sequence=['#F86E51']) 
fig_d=px.bar(daywise, x='Date', y='Deaths', color_discrete_sequence=['#FB9EC4'])

fig=make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1, 
                  subplot_titles=('<b>Potvrđeni slučajevi</b>', '<b>Smrtni slučajevi</b>'))

fig.add_trace(fig_c['data'][0], row=1, col=1) 
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=480) 
fig.show()

# **Potvrdjeni i smrtni slučajevi preko Static Colormap**


---



In [ ]:
fig_c=px.choropleth(countrywise, locations='Country', locationmode='country names',
                    color=np.log(countrywise['Confirmed']), hover_name= 'Country' ,
                    hover_data=['Confirmed'])

sample=countrywise[countrywise['Deaths']>0]
fig_d=px.choropleth(sample, locations='Country', locationmode='country names',
                    color=np.log(sample['Deaths']), hover_name= 'Country' ,
                    hover_data=['Deaths'])

fig=make_subplots(rows=1, cols=2, subplot_titles=['<b>Potvrđeni slučajevi</b>','<b>Smrtni slučajevi</b>'],
                  specs=[[{'type':'choropleth'} ,{'type':'choropleth'} ]])

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.show()

# **Analiza slučajeva top 15 država**


---




In [ ]:
top=15
fig_c=px.bar(countrywise.sort_values('Confirmed').tail(top), x='Confirmed', y='Country',
             text='Confirmed', orientation='h', color_discrete_sequence=['#F86E51'] )
fig_d=px.bar(countrywise.sort_values('Deaths').tail(top), x='Deaths', y='Country',
             text='Deaths', orientation='h', color_discrete_sequence=['#FB9EC4'] )

fig=make_subplots(rows=4, cols=2, shared_xaxes=False, horizontal_spacing=0.14,
                  vertical_spacing=.1,
                  subplot_titles=('<b>Potvrđeni slučajevi</b>', '<b>Smrtni slučajevi</b>'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=3000)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x = 'Date', y = 'Confirmed', color = 'Country', height = 600,
             title = '<b>Potvrđeni slučajevi</b>', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.update_layout(title_x=0.5)
fig.show()

# Analiza mape stabla

# Potvrdjeni slučajevi

---



In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by = 'Confirmed', ascending = False).reset_index(drop = True),
                 path = ['Country', 'Province/State'], values = 'Confirmed', height = 700,
                 title = '<b>Broj potvrđenih slučajeva</b>',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.update_layout(title_x=0.5)
fig.data[0].textinfo = 'label+text+value'
fig.show()

## **Covid19 u poređenju sa ostalim epidemijama**


---



In [ ]:
epidemics = pd.DataFrame({'epidemic' : ['COVID-19','SARS','EBOLA','MERS','H1N1'],
                          'start_year' : ['2019','2002','2013','2012','2009'],
                          'end_year' : ['2020','2004','2016','2020','2010'],
                          'confirmed' : [full_latest['Confirmed'].sum(),8422,28646,2519,7624149],
                          'death' : [full_latest['Deaths'].sum(),813,11323,866,19654],
                          })

epidemics['mortality'] = round((epidemics['death']/epidemics['confirmed'])*100,2)

epidemics.head()

In [ ]:
sample=epidemics.melt(id_vars='epidemic', value_vars=['confirmed','death','mortality'], var_name='Case', value_name = 'Value')

fig=px.bar(sample, x='epidemic', y = 'Value', color = 'epidemic', text = 'Value', facet_col = 'Case', color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize = 8, uniformtext_mode = 'hide')

fig.show() 